**This notebook is covered by the following [license](License.ipynb)  This note must not be removed**

# Installing a restricted account

Following [Armory Spinnaker Installation](https://docs.armory.io/spinnaker-install-admin-guides/manual-service-account/)

In [1]:
SOURCENAMESPACE=my-namespace
TARGETNAMESPACE=$SOURCENAMESPACE
SERVICEACCOUNT=my-serviceaccount
ROLE=my-role
ROLEBINDING=my-rolebinding

In [2]:
kubectl create namespace $TARGETNAMESPACE

namespace/my-namespace created


In [3]:
kubectl create serviceaccount --namespace=$SOURCENAMESPACE $SERVICEACCOUNT

serviceaccount/my-serviceaccount created


In [4]:
kubectl get  serviceaccount --namespace=$SOURCENAMESPACE

NAME                SECRETS   AGE
default             1         10s
my-serviceaccount   1         5s


## Create ClusterAdmin Binding

Creating a **ClusterRoleBinding** to the **ClusterRole** named **cluster-admin** is dangerous. 

Effectively the serviceaccount becomes allmighty

**It it essential to have different accounts, otherwise the cannot be removed later**



**don't**
```yaml
kubectl delete -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: ClusterRoleBinding
metadata:
  name: my-admin
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: ServiceAccount
  name: $SERVICEACCOUNT
  namespace: $SOURCENAMESPACE
```

## Namespace restricted roles

Try to use only roles with **RoleBindings** to a certain namespace. The **Role** and their **rules** can be changed later.

[Kubernetes RBAC](https://kubernetes.io/docs/reference/access-authn-authz/rbac/)

In [47]:
kubectl apply -f - <<EOF
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: ${ROLE}
  namespace: ${TARGETNAMESPACE} # Should be namespace you are granting access to
rules:
- apiGroups: [""]
  resources: ["pods","pods/logs","pods/exec"]
  verbs: ["*"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: ${ROLEBINDING}
  namespace: $TARGETNAMESPACE # Should be namespace you are granting access to
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: ${ROLE} # Should match name of Role
subjects:
- namespace: ${SOURCENAMESPACE} # Should match namespace where SA lives
  kind: ServiceAccount
  name: ${SERVICEACCOUNT} # Should match service account name, above
EOF

role.rbac.authorization.k8s.io/my-role created
rolebinding.rbac.authorization.k8s.io/my-rolebinding created


Please edit to your needs


In [6]:
# fix kube config
sed -i 's/\"\"/\"minikube\"/' /root/.kube/config
cat /root/.kube/config

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /var/lib/minikube/certs/ca.crt
    extensions:
    - extension:
        last-update: Thu, 02 Dec 2021 15:00:20 CET
        provider: minikube.sigs.k8s.io
        version: v1.23.2
      name: cluster_info
    server: https://localhost:8443
  name: "minikube"
contexts:
- context:
    cluster: "minikube"
    extensions:
    - extension:
        last-update: Thu, 02 Dec 2021 15:00:20 CET
        provider: minikube.sigs.k8s.io
        version: v1.23.2
      name: context_info
    user: "minikube"
  name: "minikube"
current-context: "minikube"
kind: Config
preferences: {}
users:
- name: "minikube"
  user:
    client-certificate: /var/lib/minikube/certs/apiserver.crt
    client-key: /var/lib/minikube/certs/apiserver.key


In [7]:
CONTEXT=$(kubectl config current-context)
echo $CONTEXT
NAMESPACE=$SOURCENAMESPACE

NEW_CONTEXT=my-context
KUBECONFIG=my-kubeconfig

minikube


In [8]:
SECRET=$(kubectl get serviceaccount ${SERVICEACCOUNT} \
  --context ${CONTEXT} \
  --namespace ${NAMESPACE} \
  -o jsonpath='{.secrets[0].name}')
TOKEN_DATA=$(kubectl get secret ${SECRET} \
  --context ${CONTEXT} \
  --namespace ${NAMESPACE} \
  -o jsonpath='{.data.token}')

TOKEN=$(echo ${TOKEN_DATA} | base64 --decode)



In [9]:
echo $TOKEN_DATA

ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNklqTlljM2g1U0ZVM2VGUnNZVkU1VVMwMVgzYzJVa3hVVVZGWFNsUkdVRkpRV0RkTlYwNVJjMUZWTXpnaWZRLmV5SnBjM01pT2lKcmRXSmxjbTVsZEdWekwzTmxjblpwWTJWaFkyTnZkVzUwSWl3aWEzVmlaWEp1WlhSbGN5NXBieTl6WlhKMmFXTmxZV05qYjNWdWRDOXVZVzFsYzNCaFkyVWlPaUp0ZVMxdVlXMWxjM0JoWTJVaUxDSnJkV0psY201bGRHVnpMbWx2TDNObGNuWnBZMlZoWTJOdmRXNTBMM05sWTNKbGRDNXVZVzFsSWpvaWJYa3RjMlZ5ZG1salpXRmpZMjkxYm5RdGRHOXJaVzR0YTNSaWFEZ2lMQ0pyZFdKbGNtNWxkR1Z6TG1sdkwzTmxjblpwWTJWaFkyTnZkVzUwTDNObGNuWnBZMlV0WVdOamIzVnVkQzV1WVcxbElqb2liWGt0YzJWeWRtbGpaV0ZqWTI5MWJuUWlMQ0pyZFdKbGNtNWxkR1Z6TG1sdkwzTmxjblpwWTJWaFkyTnZkVzUwTDNObGNuWnBZMlV0WVdOamIzVnVkQzUxYVdRaU9pSXdNV0l6TlRFek1pMDBNVE14TFRRME9UVXRZamhpTUMwNFlqZ3dNMk01WXpsbU5HRWlMQ0p6ZFdJaU9pSnplWE4wWlcwNmMyVnlkbWxqWldGalkyOTFiblE2YlhrdGJtRnRaWE53WVdObE9tMTVMWE5sY25acFkyVmhZMk52ZFc1MEluMC56UF9pMDJDUmc0eDl5MEh5OExuS0JTRDFuZjA0aWU5X2tDRnhCUDR6Y2JFeVBGX0VSMTJlR0VhSS1LVlAwQVpVS3htZnRhS0dJT0ttNHk3T2Jjc1pwZzE0OFptN3FlV0pac0pOXzAyVjFJb1lTbEFMWjRFQ2trTDFYNGk0SXhNTm5MWHFJZ0dDT1hy

## **Create a dedicated kubeconfig**

Starting with a full copy

In [10]:
echo $KUBECONFIG

my-kubeconfig


In [11]:

kubectl config view --raw > ${KUBECONFIG}.full.tmp

Switch working context to correct context

In [12]:
kubectl --kubeconfig ${KUBECONFIG}.full.tmp config use-context ${CONTEXT}

Switched to context "minikube".


Minify and flatten the config

In [13]:
kubectl --kubeconfig ${KUBECONFIG}.full.tmp \
  config view --flatten --minify > ${KUBECONFIG}.tmp

In [14]:
cat ${KUBECONFIG}.tmp

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ETXdNVEE1TlRRMU1Gb1hEVE14TURJeU9EQTVOVFExTUZvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTkh5Cm1SRm1qaG0wdWpKRXd4ZXp2VWZvL0o2Q1Mvbkk3MS9rQlZxVEVTY3ZoZFpkYjdUazl3T1Bueng3VW9UQUIwZXoKZ3AxUjdnVFBnZnJSNGNyKy9Lc08vWExydk9CNEFoOXM0RE1lL1B5OTVyZE1iNmlDUytZMDV1VldkQ3R0bGhrSwpjM3lmblhPS29teXFaVUw0dzJLTFQza1I2clBHVXZ6M3RMYU9lYlpYc2RYNklqZXBaOHVROXZkYldXYVVVS25uCk5jUHdLc3VCNmNra1hibnBmaXRXSWovVnEwbFdZWU5OOXVJZjg0TWFqY2ZjS2twRUpoY2g4SHd2aWdtZGRiWHkKL2hnNXd0VGF3RCtUUWp5d09MVFpIMkg5bEVodGNQMS9PeE9nOU56WitaQW1CcDZFelFJbmFJMCt0YVM2WW1FbQppQ0RzUVlEYnV0bXpFWnliVEJjQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJRY004c25IMk10R1k3dU55UlZC

Rename context

In [15]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  rename-context ${CONTEXT} ${NEW_CONTEXT}

Context "minikube" renamed to "my-context".


Set the user credentials for the user created


In [16]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  set-credentials ${CONTEXT}-${NAMESPACE}-token-user \
  --token ${TOKEN}

User "minikube-my-namespace-token-user" set.


Set context to use token user

In [17]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  set-context ${NEW_CONTEXT} --user ${CONTEXT}-${NAMESPACE}-token-user

Context "my-context" modified.


Set context to correct namespace

In [18]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  set-context ${NEW_CONTEXT} --namespace ${NAMESPACE}

Context "my-context" modified.


Flatten/minify kubeconfig


In [19]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  view --flatten --minify > ${KUBECONFIG}

cleanup

In [ ]:
rm ${KUBECONFIG}.full.tmp
rm ${KUBECONFIG}.tmp

In [48]:
kubectl --kubeconfig ${KUBECONFIG} get pods

NAME     READY   STATUS    RESTARTS   AGE
nginx2   1/1     Running   0          26m


In [49]:
kubectl --kubeconfig ${KUBECONFIG} get pods -A

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "pods" in API group "" at the cluster scope


: 1

In [51]:
kubectl --kubeconfig ${KUBECONFIG} get svc

Error from server (Forbidden): services is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "services" in API group "" in the namespace "my-namespace"


: 1

In [22]:
cat ${KUBECONFIG}

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ETXdNVEE1TlRRMU1Gb1hEVE14TURJeU9EQTVOVFExTUZvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTkh5Cm1SRm1qaG0wdWpKRXd4ZXp2VWZvL0o2Q1Mvbkk3MS9rQlZxVEVTY3ZoZFpkYjdUazl3T1Bueng3VW9UQUIwZXoKZ3AxUjdnVFBnZnJSNGNyKy9Lc08vWExydk9CNEFoOXM0RE1lL1B5OTVyZE1iNmlDUytZMDV1VldkQ3R0bGhrSwpjM3lmblhPS29teXFaVUw0dzJLTFQza1I2clBHVXZ6M3RMYU9lYlpYc2RYNklqZXBaOHVROXZkYldXYVVVS25uCk5jUHdLc3VCNmNra1hibnBmaXRXSWovVnEwbFdZWU5OOXVJZjg0TWFqY2ZjS2twRUpoY2g4SHd2aWdtZGRiWHkKL2hnNXd0VGF3RCtUUWp5d09MVFpIMkg5bEVodGNQMS9PeE9nOU56WitaQW1CcDZFelFJbmFJMCt0YVM2WW1FbQppQ0RzUVlEYnV0bXpFWnliVEJjQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJRY004c25IMk10R1k3dU55UlZC

In [ ]:
kubectl --kubeconfig ${KUBECONFIG} get pods 

In [23]:
kubectl --kubeconfig ${KUBECONFIG}  run nginx2 --image=nginx --port=80

pod/nginx2 created


In [24]:
kubectl get pods -A

NAMESPACE       NAME                                        READY   STATUS              RESTARTS      AGE
default         opa-69c9967959-6b6kl                        0/1     ContainerCreating   0             46m
ingress-nginx   ingress-nginx-admission-create--1-r9glx     0/1     Completed           0             48m
ingress-nginx   ingress-nginx-admission-patch--1-jmxgb      0/1     Completed           0             48m
ingress-nginx   ingress-nginx-controller-69bdbc4d57-n9hnq   1/1     Running             0             48m
kube-system     coredns-78fcd69978-txbhl                    1/1     Running             0             50m
kube-system     etcd-minikube                               1/1     Running             0             50m
kube-system     kube-apiserver-minikube                     1/1     Running             0             50m
kube-system     kube-controller-manager-minikube            1/1     Running             0             50m
kube-system     kube-proxy-5hs49              

In [44]:
kubectl --kubeconfig ${KUBECONFIG} get pods

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "pods" in API group "" in the namespace "my-namespace"


: 1

In [45]:
kubectl --kubeconfig ${KUBECONFIG} get secrets

Error from server (Forbidden): secrets is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "secrets" in API group "" in the namespace "my-namespace"


: 1

In [27]:
kubectl --kubeconfig ${KUBECONFIG} --namespace default get deployment

Error from server (Forbidden): deployments.apps is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "deployments" in API group "apps" in the namespace "default"


: 1

In [28]:
kubectl --kubeconfig ${KUBECONFIG}  --namespace default run nginx2 --image=nginx --port=80 

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot create resource "pods" in API group "" in the namespace "default"


: 1

In [29]:
cat  ${KUBECONFIG} 

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ETXdNVEE1TlRRMU1Gb1hEVE14TURJeU9EQTVOVFExTUZvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTkh5Cm1SRm1qaG0wdWpKRXd4ZXp2VWZvL0o2Q1Mvbkk3MS9rQlZxVEVTY3ZoZFpkYjdUazl3T1Bueng3VW9UQUIwZXoKZ3AxUjdnVFBnZnJSNGNyKy9Lc08vWExydk9CNEFoOXM0RE1lL1B5OTVyZE1iNmlDUytZMDV1VldkQ3R0bGhrSwpjM3lmblhPS29teXFaVUw0dzJLTFQza1I2clBHVXZ6M3RMYU9lYlpYc2RYNklqZXBaOHVROXZkYldXYVVVS25uCk5jUHdLc3VCNmNra1hibnBmaXRXSWovVnEwbFdZWU5OOXVJZjg0TWFqY2ZjS2twRUpoY2g4SHd2aWdtZGRiWHkKL2hnNXd0VGF3RCtUUWp5d09MVFpIMkg5bEVodGNQMS9PeE9nOU56WitaQW1CcDZFelFJbmFJMCt0YVM2WW1FbQppQ0RzUVlEYnV0bXpFWnliVEJjQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJRY004c25IMk10R1k3dU55UlZC

In [30]:
kubectl describe serviceaccount -n  my-namespace

Name:                default
Namespace:           my-namespace
Labels:              <none>
Annotations:         <none>
Image pull secrets:  <none>
Mountable secrets:   default-token-552fm
Tokens:              default-token-552fm
Events:              <none>


Name:                my-serviceaccount
Namespace:           my-namespace
Labels:              <none>
Annotations:         <none>
Image pull secrets:  <none>
Mountable secrets:   my-serviceaccount-token-ktbh8
Tokens:              my-serviceaccount-token-ktbh8
Events:              <none>


In [31]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i get pod --namespace default

no


: 1

In [32]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i create pod --namespace default

no


: 1

In [53]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i create pod --namespace $TARGETNAMESPACE

yes


In [39]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i create clusterroles

no


: 1

In [54]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i create roles

no


: 1

In [55]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i --list

Resources                                       Non-Resource URLs                     Resource Names   Verbs
pods/exec                                       []                                    []               [*]
pods/logs                                       []                                    []               [*]
pods                                            []                                    []               [*]
selfsubjectaccessreviews.authorization.k8s.io   []                                    []               [create]
selfsubjectrulesreviews.authorization.k8s.io    []                                    []               [create]
                                                [/.well-known/openid-configuration]   []               [get]
                                                [/api/*]                              []               [get]
                                                [/api]                                []               [get]
                   

In [52]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i create deployments

no


: 1

In [34]:
kubectl  --kubeconfig ${KUBECONFIG} cluster-info


To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
Error from server (Forbidden): services is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "services" in API group "" in the namespace "kube-system"


: 1

In [35]:
kubectl  cluster-info

Kubernetes control plane is running at https://localhost:8443
CoreDNS is running at https://localhost:8443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [ ]:
kubectl  --kubeconfig ${KUBECONFIG} cluster-info

# Graphical View

In [59]:
go install -v github.com/mhausenblas/rback@latest

go: downloading github.com/mhausenblas/rback v0.3.0
go: downloading github.com/emicklei/dot v0.10.0
go: downloading github.com/mhausenblas/kubecuddler v0.0.0-20181012110128-5836f3e4e7d0
github.com/mhausenblas/kubecuddler
github.com/emicklei/dot
github.com/mhausenblas/rback


In [70]:
kubectl get sa,roles -n my-namespace -o json |\
rback  -ignore-prefixes none > minikube.dot

/usr/bin/kubectl get sa --all-namespaces --output json
/usr/bin/kubectl get roles --all-namespaces --output json
/usr/bin/kubectl get rolebindings --all-namespaces --output json
/usr/bin/kubectl get clusterroles --output json
/usr/bin/kubectl get clusterrolebindings --output json


In [68]:
apt install graphviz -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.42.2-3build2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [71]:
dot -Tpng -Grankdir=LR minikube.dot -o minikube.png

![Minikube](minikube.png)

In [63]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i --list

Resources                                       Non-Resource URLs                     Resource Names   Verbs
pods/exec                                       []                                    []               [*]
pods/logs                                       []                                    []               [*]
pods                                            []                                    []               [*]
selfsubjectaccessreviews.authorization.k8s.io   []                                    []               [create]
selfsubjectrulesreviews.authorization.k8s.io    []                                    []               [create]
                                                [/.well-known/openid-configuration]   []               [get]
                                                [/api/*]                              []               [get]
                                                [/api]                                []               [get]
                   

In [64]:
kubectl describe clusterrole.rbac  | grep --color=always -E '(create)'

  rolebindings.rbac.authorization.k8s.io          []                 []              [create delete deletecollection get list patch update watch]
  roles.rbac.authorization.k8s.io                 []                 []              [create delete deletecollection get list patch update watch]
  configmaps                                      []                 []              [create delete deletecollection patch update get list watch]
  events                                          []                 []              [create delete deletecollection patch update get list watch]
  persistentvolumeclaims                          []                 []              [create delete deletecollection patch update get list watch]
  pods                                            []                 []              [create delete deletecollection patch update get list watch]
  replicationcontrollers/scale                    []                 []              [create delete deletecollection patch u

In [65]:
kubectl get sa,roles,clusterroles,clusterrolebindings -n my-namespace -o json | \
rback vis --include-namespaces=my-namespace --outformat dot > my-namespace-rbac.dot


/usr/bin/kubectl get sa --all-namespaces --output json
/usr/bin/kubectl get roles --all-namespaces --output json
/usr/bin/kubectl get rolebindings --all-namespaces --output json
/usr/bin/kubectl get clusterroles --output json
/usr/bin/kubectl get clusterrolebindings --output json


In [66]:
dot -Tpng -Grankdir=LR  my-namespace-rbac.dot -o  my-namespace-rbac.png

![ my-namespace-rbac.png](my-namespace-rbac.png)

In [ ]:
kubectl auth can-i --list 

In [ ]:
kubectl get sa,roles,clusterroles,clusterrolebindings  -o json | \
rback vis --include-namespaces=my-namespace --outformat dot > defaut-rbac.dot

In [ ]:
dot -Tpng -Grankdir=LR  defaut-rbac.dot -o  default-rbac.png

In [ ]:
ls *png

In [ ]:
kubectl  -n my-namespace get sa -o yaml

In [ ]:
echo $KUBECONFIG

In [ ]:
kubectl get rolebinding -A -o yaml

In [ ]:
# cleanup
#rm minikube.dot minikube.png my-namespace-rbac.dot my-namespace-rbac.png my-kubeconfig
kubectl config current-context

In [ ]:
go get -v github.com/thomasfricke/rbac-tool